In [3]:
import os.path as osp
import time

import torch
import torch.nn.functional as F
from sklearn.linear_model import LogisticRegression

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import GraphSAGE

dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

train_loader = LinkNeighborLoader(
    data,
    batch_size=256,
    shuffle=True,
    neg_sampling_ratio=1.0,
    num_neighbors=[10, 10],
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device, 'x', 'edge_index')

model = GraphSAGE(
    data.num_node_features,
    hidden_channels=64,
    num_layers=2,
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


def train():
    model.train()

    total_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        print(batch)
        optimizer.zero_grad()
        h = model(batch.x, batch.edge_index)
        h_src = h[batch.edge_label_index[0]]
        h_dst = h[batch.edge_label_index[1]]
        pred = (h_src * h_dst).sum(dim=-1)
        loss = F.binary_cross_entropy_with_logits(pred, batch.edge_label)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.size(0)

    return total_loss / data.num_nodes


@torch.no_grad()
def test():
    model.eval()
    out = model(data.x, data.edge_index).cpu()

    clf = LogisticRegression()
    clf.fit(out[data.train_mask], data.y[data.train_mask])

    val_acc = clf.score(out[data.val_mask], data.y[data.val_mask])
    test_acc = clf.score(out[data.test_mask], data.y[data.test_mask])

    return val_acc, test_acc


times = []
for epoch in range(1, 51):
    start = time.time()
    loss = train()
    val_acc, test_acc = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Val: {val_acc:.4f}, Test: {test_acc:.4f}')
    times.append(time.time() - start)
print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")

Data(x=[2395, 1433], edge_index=[2, 7840], y=[2395], train_mask=[2395], val_mask=[2395], test_mask=[2395], n_id=[2395], e_id=[7840], input_id=[256], edge_label_index=[2, 512], edge_label=[512])
Data(x=[2378, 1433], edge_index=[2, 7880], y=[2378], train_mask=[2378], val_mask=[2378], test_mask=[2378], n_id=[2378], e_id=[7880], input_id=[256], edge_label_index=[2, 512], edge_label=[512])
Data(x=[2403, 1433], edge_index=[2, 7720], y=[2403], train_mask=[2403], val_mask=[2403], test_mask=[2403], n_id=[2403], e_id=[7720], input_id=[256], edge_label_index=[2, 512], edge_label=[512])
Data(x=[2374, 1433], edge_index=[2, 7897], y=[2374], train_mask=[2374], val_mask=[2374], test_mask=[2374], n_id=[2374], e_id=[7897], input_id=[256], edge_label_index=[2, 512], edge_label=[512])
Data(x=[2401, 1433], edge_index=[2, 7817], y=[2401], train_mask=[2401], val_mask=[2401], test_mask=[2401], n_id=[2401], e_id=[7817], input_id=[256], edge_label_index=[2, 512], edge_label=[512])
Data(x=[2418, 1433], edge_inde

[E thread_pool.cpp:110] Exception in thread pool task: mutex lock failed: Invalid argument


KeyboardInterrupt: 